In [4]:
import os
import sys

import gymnasium as gym
import torch

sys.path.append(os.path.abspath(".."))

from rlib.algorithms.reinforce import reinforce
from rlib.common.policies import DiscreteStochasticMlpPolicy

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
env = gym.make("CartPole-v1")

In [6]:
discrete = True

input_size = env.observation_space.shape[0]

if discrete:
    output_size = env.action_space.n
else:
    output_size = 2 * env.action_space.shape[0]

print(input_size, output_size)

4 2


In [7]:
policy = DiscreteStochasticMlpPolicy(input_size, output_size)
optimizer = torch.optim.Adam(policy.parameters(), lr=3e-4)

In [8]:
reinforce(env, policy, optimizer)

c:\Users\никита\coding\rl\rlib\rlib\common\buffer.py:29: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:264.)
  "observations": torch.tensor(self.observations, dtype=torch.float32),


steps_n: 1012
mean_trajectory_rewards: 13.0
mean_trajectory_length: 13.0
steps_n: 2010
mean_trajectory_rewards: 14.0
mean_trajectory_length: 14.0
steps_n: 3006
mean_trajectory_rewards: 11.0
mean_trajectory_length: 11.0
steps_n: 4017
mean_trajectory_rewards: 66.0
mean_trajectory_length: 66.0
steps_n: 5009
mean_trajectory_rewards: 54.0
mean_trajectory_length: 54.0
steps_n: 6004
mean_trajectory_rewards: 33.0
mean_trajectory_length: 33.0
steps_n: 7066
mean_trajectory_rewards: 70.0
mean_trajectory_length: 70.0
steps_n: 8013
mean_trajectory_rewards: 34.0
mean_trajectory_length: 34.0
steps_n: 9031
mean_trajectory_rewards: 85.0
mean_trajectory_length: 85.0
steps_n: 10043
mean_trajectory_rewards: 59.0
mean_trajectory_length: 59.0
steps_n: 11075
mean_trajectory_rewards: 81.0
mean_trajectory_length: 81.0
steps_n: 12027
mean_trajectory_rewards: 33.0
mean_trajectory_length: 33.0
steps_n: 13072
mean_trajectory_rewards: 108.0
mean_trajectory_length: 108.0
steps_n: 14013
mean_trajectory_rewards: 32.0
